GOAL: Export and download data from ArcGIS Online/Portal Hosted Feature Service

In [ ]:
import arcpy
import os
import time
import zipfile
from arcgis.gis import GIS
from getpass import getpass
from zipfile import ZipFile
from zipfile import ZipInfo

In [ ]:
downloadPath = 'c://users//friendde//Downloads//'
downloadPostfix = '_fgdb.zip'

In [ ]:
response = input(f'Do you want to use the active portal? {(arcpy.GetActivePortalURL())} [y or n]')
if response.upper() in "YES":
    #arcpy.SignInToPortal(input("Enter portal address "), username=(input("Enter User Name ")), password=(getpass()))
    gis = GIS(arcpy.GetActivePortalURL(), username=input("Enter User Name "), password=(getpass()))
else:
    arcpy.SignInToPortal(input("Enter portal address "), username=(input("Enter User Name ")), password=getpass())
    #gis = GIS(input("Enter portal address "), input("Enter User Name "), getpass())
portal_desc = arcpy.GetPortalDescription()
print(f'Portal Name {portal_desc["portalName"]}\nConnected to {portal_desc["name"]} as user {portal_desc["user"]["username"]}')

In [ ]:
# search and list all items hosted on the portal
query=f'owner:{portal_desc["user"]["username"]}'
#query=''
itemType="Feature Layer"
sortField="title"
sortOrder="asc"
# default max__items is 10
maxItems=100
searchResult = gis.content.search(query,itemType,sortField,sortOrder,maxItems)
for i in range(len(searchResult)):
    display(searchResult[i])
    downloadResult = input(f'Download {(searchResult[i].title)} [Y,N]')
    if downloadResult.upper() in "YES":
        sdItem = searchResult[i]
        print(sdItem.title, sdItem.id, sdItem.modified, sdItem.access)
        break
if downloadResult.upper() not in "YES":
    print(f'All Feature Layers hosted on {portal_desc["portalName"]} were presented')

data_item = gis.content.get(sdItem.id)
data_item

In [ ]:
fgdbItem = gis.content.get(sdItem.id).export(sdItem.title + '_fgdb', 'File Geodatabase', parameters=None, wait='True')

## intial output is a 1 KB file that is not a valid zipped file.
## The while statement waits for a valid zipped file to be created.
#full path to the zipped file
zipfullpath=os.path.join(downloadPath,sdItem.title + '_fgdb.zip')
while  zipfile.is_zipfile(zipfullpath)==False:
    print("Waiting for valid zip file")
    gis.content.get(fgdbItem['exportItemId']).download(save_path = downloadPath)
    time.sleep(5)
zf = ZipFile(os.path.join(downloadPath,sdItem.title + '_fgdb.zip'))
zf.extractall(path=downloadPath + sdItem.title + '_fgdb')

In [ ]:
file_list = os.listdir(downloadPath + sdItem.title + '_fgdb')
file_list